In [1]:
from selenium import webdriver
from selenium.common.exceptions import ElementNotVisibleException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time
import re
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta as time_ago
from dateutil.parser import parser
import pandas as pd
import numpy as np
import requests
import string
import random
import os

In [2]:
def make_folder(fn):
    """
    Creates a folder fn if it doesn't exist
    """
    if not os.path.exists(fn):
        os.makedirs(fn)

In [3]:
def make_nested_folders(folder_structure=None):
    """
    folder_structure (dict):  {'outer_folder1': ['subfolders', 'of', 'outer_folder1']}
        Creates (in current dir):
            outer_folder1/
                      subfolders/
                      of/
                      outer_folder1/
    """
    if folder_structure is not None:
        for parent_folder, children in folder_structure.items():
            make_folder(parent_folder)
            for child_folder in children:
                make_folder(parent_folder + '/' + child_folder)

In [4]:
def error_message(place_of_error):
    print('something went wrong when processing', place_of_error)

In [5]:
def download_html(browser, asin, page_number, reason=''):
    """
    Download's page html just in case live parsing messes up (a backup)
    """
    source = browser.page_source
    html_fn = 'vendor-profiles/' + asin + '/html/' + str(page_number) + '_' + reason + '.html'
    with open(html_fn, 'w', encoding='utf-8') as handler:
        handler.write(source)

In [6]:
def nap(lb=0, ub=1):
    """
    sleep scrape for a randomly generated time in seconds,
    set a lower bound (lb) and upper (ub) for the RGN.
    meant to hopefully avoid robot checks.
    """
    time.sleep(random.uniform(lb, ub))

In [7]:
def pause_up_up(browser, current_height):
    """
    another function to avoid robot checks-
    scrolls up a random amount of pixels 3-4 times with micronaps fit in.
    """
    y = current_height
    for i in range(random.randint(3,4)):
        y_wiggle = y - random.randint(80,140)
        browser.execute_script("window.scrollTo(0, %s)" % y_wiggle)
        y = y_wiggle
        nap(.3, 1.15)

In [8]:
def control_scroll(browser):
    scroll_heights = [344, 809]
    nap(.75, 2.2)
    for page_y in scroll_heights:
        y_wiggle = page_y + random.randint(-150,100)
        browser.execute_script("window.scrollTo(0, %s)" % y_wiggle)
        nap(.3, .8)
        if random.randint(1,2) > 1:
            nap(1.4, 2.6)
        else:
            pause_up_up(browser, y_wiggle)
            nap(.2, .6)

In [9]:
def asin_vendors_url(asin):
    url = 'https://www.amazon.com/gp/offer-listing/' + asin + '/ref=olp_f_new?ie=UTF8&f_all=true&f_new=true'
    return url

In [10]:
def parse_shipping(shipping_soup):
    amzn_fulfill = shipping_soup.find_all('div', {'class': 'olpBadgeContainer'})
    if len(amzn_fulfill) == 0:  # no amazon fulfillment badge
        return False
    else:
        return True

In [11]:
def parse_price(price_soup):
    price_text = price_soup.find_all('span', {"class" : "a-size-large a-color-price olpOfferPrice a-text-bold"})
    if len(price_text) == 0:
        error_message('price text')
        price_text = None
    else:
        price = price_text[0].text.strip()[1:]
        try:
            price = float(price)
        except:
            error_message('price cast to float')
            
    prime_soup = price_soup.find_all('i', {'class': 'a-icon a-icon-prime'})
    if len(prime_soup) == 0:
        is_prime = False
    else:
        is_prime = True
    return price, is_prime

In [12]:
def parse_seller_details(seller_soup):
    name_soup = seller_soup.find_all('span', {'class': 'a-size-medium a-text-bold'})
    # if no vendor details found:
    if len(name_soup) == 0:
        # if there's an amazon badge, then amazon is the vendor
        if len(seller_soup.find_all('img', {'alt': 'Amazon.com'})) > 0:
            seller_name, vendor_url = 'Amazon', None  # amazon doesn't publish their vendor stats
            pct_positive, over_time_span, num_ratings = None, None, None
            return seller_name, vendor_url, pct_positive, over_time_span, num_ratings
        # if no amazon badge, then something went wrong
        else:
            error_message('seller name')
            seller_name, vendor_url = None, None
    else:
        seller_name = name_soup[0].get_text().strip()
        vendor_url = name_soup[0].findChildren()[0]['href']
    try:
        seller_details_soup = seller_soup.find_all('p', {'class': 'a-spacing-small'})
        seller_reputation = seller_details_soup[0].text.split('\n')[2]
    except:
        error_message('seller_details')
        return seller_name, vendor_url, None, None, None
    try:
        pct_positive = seller_reputation.split('%')[0].strip()
    except:
        error_message('pct_positive')
        pct_positive = None
    try:
        over_time_span = seller_reputation.split('.')[0].split('past')[1].strip()
    except:
        print('No vendor reputation time span found ("97% over 12 months...")')
        over_time_span = None
    try:
        num_ratings = int(seller_reputation.split('(')[1].split(' total')[0].replace(",", ""))
    except:
        error_message('num_ratings')
        num_ratings = None
    return seller_name, vendor_url, pct_positive, over_time_span, num_ratings

In [13]:
def parse_vendor(vendor_soup):
    price_soup = vendor_soup.find_all('div', {'class': 'a-column a-span2 olpPriceColumn'})
    if len(price_soup) == 0:
        error_message('price')
        price, is_prime = None, None
    else:
        price, is_prime = parse_price(price_soup[0])
    shipping_soup = vendor_soup.find_all('div', {'class': 'a-column a-span3 olpDeliveryColumn'})
    if len(shipping_soup) == 0:
        error_message('shipping')
        is_amzn_fulf = None
    else:
        is_amzn_fulf = parse_shipping(shipping_soup[0])
    seller_soup = vendor_soup.find_all('div', {'class': 'a-column a-span2 olpSellerColumn'})
    if len(seller_soup) == 0:
        error_message('seller')
        seller_name, vendor_url, pct_positive, over_time_span, num_ratings = None, None, None, None, None
    else:
        seller_name, vendor_url, pct_positive, over_time_span, num_ratings = parse_seller_details(seller_soup[0])
    
    vendor_data = {
        'price': price,
        'is_prime': is_prime,
        'is_amzn_fulf': is_amzn_fulf,
        'name': seller_name,
        'vendor_url': vendor_url,
        'pct_positive': pct_positive,
        'over_time_span': over_time_span,
        'num_ratings': num_ratings
    }
    return vendor_data

In [14]:
def is_last_page(soup):
    greyed_out_next = soup.find_all('ul', {'class': "a-pagination"})[0].find_all('li', {'class': 'a-disabled a-last'})
    if len(greyed_out_next) == 0:
        return False
    return True

In [15]:
def iter_pages(browser):
    soup = bs(browser.page_source, 'lxml')
    page_buttons = soup.find_all('ul', {'class': "a-pagination"})

    only_one_page = len(page_buttons) == 0
    if only_one_page:
        return True
    on_last_page = is_last_page(soup)
    if on_last_page:
        return True
    else:
        href = page_buttons[0].find_all('li', {'class': 'a-last'})[0].findChildren()[0]['href']
        browser.get('https://www.amazon.com' + href)
        control_scroll(browser)
        return False

In [16]:
def scrape_page(browser, asin):
    soup = bs(browser.page_source, 'lxml')
    vendor_blocks = soup.find_all('div', {'class': 'a-row a-spacing-mini olpOffer'})
    page_vendor_data = []
    for vendor_block in vendor_blocks:
        try:
            vendor_data = parse_vendor(vendor_block)
        except:
            error_message(asin + ' scrape')
            continue
        vendor_data['asin'] = asin
        page_vendor_data.append(vendor_data)
    return page_vendor_data

In [17]:
def scrape_asin_vendors(asin, browser=None, path_to_chromedriver=None):
    if browser == None:
        if not path_to_chromedriver:
            path_to_chromedriver = os.path.abspath('chromedriver')
        browser = webdriver.Chrome(path_to_chromedriver)
    make_folder('vendors')
    make_nested_folders({'vendors/' + asin: ['html', 'raw', 'processed']})
    all_vendor_data = []
    start_url = asin_vendors_url(asin)
    browser.get(start_url)
    nap(.85,1.5)
    page_num = 1
    on_last_page = False
    while not on_last_page:
        all_vendor_data.extend(scrape_page(browser, asin))
        download_html(browser, asin, page_num)
        on_last_page = iter_pages(browser)
        page_num += 1
    try:
        df = pd.DataFrame(all_vendor_data)
        df.to_csv('vendors/' + asin + '/raw/raw.csv', index=False)
    except:
        error_message('saving scrape for ' + asin)
    return browser, all_vendor_data

In [18]:
def scrape_many_asins(list_of_asins, path_to_chromedriver=None):
    browser = None
    all_asins_data = []
    for asin in list_of_asins:
        print('\n\n' + asin + '\n')
        browser, all_vendor_data = scrape_asin_vendors(asin, browser)
        all_asins_data.extend(all_vendor_data)
    try:
        df = pd.DataFrame(all_vendor_data)
        df.to_csv('all-scrape2.csv', index=False)
    except:
        error_message('saving scrape for ' + asin)
    return browser

In [19]:
def google_vendor_id(browser, vendor_id):
    browser.get('https://www.google.com/search?q=' + vendor_id)
    soup = bs(browser.page_source, 'lxml')
    search_results = soup.find_all('div', {'class': 'rc'})
    for result in search_results:
        if 'Amazon.com Seller Profile' in result.text or '30 days, 90 days' in result.text:
            link = result.find_all('h3', {'class': 'r'})[0].findChildren()[0]['href']
            return link
    return None

In [20]:
#unfinished
def iter_vendor_review_page(browser):
    browser.find_elements_by_id('feedback-next-link')[0].click()
    nap(1.5,2.5)
    while True:
        try:
            b.find_elements_by_id('feedback-next-link')[0].click()
            nap(1.5,2.5)
        except ElementNotVisibleException:
            print('nv')
            break
        except IndexError:
            print('ie')
            break
        except:
            print('l')
            break

In [21]:
def get_feedback_table_data(soup):
    table_data = {
        'pos_30d': None,
        'pos_90d': None,
        'pos_1yr': None,
        'pos_lifetime': None,
        'neut_30d': None,
        'neut_90d': None,
        'neut_1yr': None,
        'neut_lifetime': None,
        'neg_30d': None,
        'neg_90d': None,
        'neg_1yr': None,
        'neg_lifetime': None,
        'ct_30d': None,
        'ct_90d': None,
        'ct_1yr': None,
        'ct_lifetime': None,
    }
    table = soup.find_all('table', {'id': 'feedback-summary-table'})
    if len(table) == 0:
        error_message('feedback_table_data')
        return table_data
    table = soup.find_all('table', {'id': 'feedback-summary-table'})[0]
    rows = table.find_all('td', {'class': 'a-text-right'})
    if len(rows) == 0:
        error_message('parsing table')
        return table_data

    for key, row in zip(table_data.keys(), rows):
        row_elements = row.findChildren()
        value = row_elements[0].text.strip()
        table_data[key] = value
    return table_data

In [22]:
def parse_vendor_reviews(soup, vendor_id, vendor_review_data=[]):
    review_rows = soup.find_all('tr', {'class': 'feedback-row'})
    return vendor_review_data

In [23]:
def scrape_vendor_profile(vendor_url, browser=None, use_url=False, path_to_chromedriver=None):
#     full_url = lambda x: 
    vendor_id = url_to_vendor_id(vendor_url)
    if browser == None:
        if not path_to_chromedriver:
            path_to_chromedriver = os.path.abspath('chromedriver')
        browser = webdriver.Chrome(path_to_chromedriver)
    make_folder('vendor-profiles')
    make_nested_folders({'vendor-profiles/' + vendor_id: ['html', 'raw', 'processed']})
    all_profile_data = []
    if use_url:
        browser.get('https://wwww.amazon.com' + vendor_url)
    else:
        profile_url = google_vendor_id(browser, vendor_id)
        nap(.5, 1.25)
        browser.get(profile_url)
    nap(.85,1.5)
    soup = bs(browser.page_source, 'lxml')
    download_html(browser, vendor_id, 1)
    control_scroll(browser)
    feedback_table_data = get_feedback_table_data(soup)
    feedback_table_data['vendor_id'] = vendor_id
    print(feedback_table_data)
    df = pd.DataFrame([feedback_table_data])
    df.to_csv('vendor-profiles/' + vendor_id + '/raw/raw.csv', index=False)
    return browser, feedback_table_data

In [24]:
def scrape_many_vendors(vendor_urls, path_to_chromedriver=None):
    browser = None
    all_vendor_data = []
    for vendor_url in vendor_urls:
        print('\n' + vendor_url)
        try:
            browser, vendor_data = scrape_vendor_profile(vendor_url, browser)
            all_vendor_data.extend(vendor_data)
        except:
            try:
                browser, vendor_data = scrape_vendor_profile(vendor_url, browser, use_url=True)
            except:
                error_message('processing vendor', vendor_url)
    try:
        df = pd.DataFrame(all_vendor_data)
        df.to_csv('data/vendor-profiles/all-scrape.csv', index=False)
        return browser, all_vendor_data
    except:
        error_message('saving scrape')
        return browser, all_vendor_data

# Aggregation functions

In [25]:
def aggregate_data(path='vendors'):
    asin_folders = os.listdir(path)
    all_asin_data = []
    all_error_asins = []
    print(len(asin_folders))
    for folder in asin_folders:
        path_to_raw = path + '/' + folder + '/raw'
        if 'raw.csv' in os.listdir(path_to_raw):
            try:
                all_asin_data.append(pd.read_csv(path_to_raw + '/raw.csv'))
            except:
                print('error', folder)
                all_error_asins.append(folder)
    return all_asin_data, all_error_asins

In [26]:
def url_to_vendor_id(vendor_url):
    if type(vendor_url) == str:
        return vendor_url.split('&seller=')[-1]
    else:
        print(vendor_url)
        return None

In [27]:
def fix_pct_pos(val):
    if type(val) == str:
        try:
            return float(val.strip())
        except:
            return None
    try:
        return float(val)
    except:
        return None

In [37]:
test = '/gp/aag/main/ref=olp_merch_name_6?ie=UTF8&asin=B00PWK8YIS&isAmazonFulfilled=0&seller=A2L8DRBILLKUMV'

In [31]:
worst_to_best[1]

'/gp/aag/main/ref=olp_merch_name_2?ie=UTF8&asin=B06XTVQN94&isAmazonFulfilled=0&seller=A3CAAQNB3F7JZA'

In [38]:
b, v = scrape_many_vendors(worst_to_best)
# b,v = scrape_vendor_profile(test)


/gp/aag/main/ref=olp_merch_name_6?ie=UTF8&asin=B00PWK8YIS&isAmazonFulfilled=0&seller=A2L8DRBILLKUMV
{'pos_30d': '-', 'pos_90d': '-', 'pos_1yr': '33', 'pos_lifetime': '54', 'neut_30d': '-', 'neut_90d': '-', 'neut_1yr': '0', 'neut_lifetime': '15', 'neg_30d': '-', 'neg_90d': '-', 'neg_1yr': '67', 'neg_lifetime': '31', 'ct_30d': '0', 'ct_90d': '0', 'ct_1yr': '3', 'ct_lifetime': '13', 'vendor_id': 'A2L8DRBILLKUMV'}

/gp/aag/main/ref=olp_merch_name_2?ie=UTF8&asin=B06XTVQN94&isAmazonFulfilled=0&seller=A3CAAQNB3F7JZA
something went wrong when processing feedback_table_data
{'pos_30d': None, 'pos_90d': None, 'pos_1yr': None, 'pos_lifetime': None, 'neut_30d': None, 'neut_90d': None, 'neut_1yr': None, 'neut_lifetime': None, 'neg_30d': None, 'neg_90d': None, 'neg_1yr': None, 'neg_lifetime': None, 'ct_30d': None, 'ct_90d': None, 'ct_1yr': None, 'ct_lifetime': None, 'vendor_id': 'A3CAAQNB3F7JZA'}

/gp/aag/main/ref=olp_merch_name_6?ie=UTF8&asin=B004EF5VVA&isAmazonFulfilled=0&seller=A3EA4OFUZKHSR6
{'

{'pos_30d': '78', 'pos_90d': '80', 'pos_1yr': '78', 'pos_lifetime': '90', 'neut_30d': '0', 'neut_90d': '7', 'neut_1yr': '4', 'neut_lifetime': '2', 'neg_30d': '22', 'neg_90d': '13', 'neg_1yr': '18', 'neg_lifetime': '8', 'ct_30d': '27', 'ct_90d': '60', 'ct_1yr': '129', 'ct_lifetime': '504', 'vendor_id': 'A3E6IRIG1001ON'}

/gp/aag/main/ref=olp_merch_name_1?ie=UTF8&asin=B071HWT4BC&isAmazonFulfilled=0&seller=ABOS4QKXWWCIJ
{'pos_30d': '75', 'pos_90d': '73', 'pos_1yr': '78', 'pos_lifetime': '93', 'neut_30d': '0', 'neut_90d': '0', 'neut_1yr': '0', 'neut_lifetime': '3', 'neg_30d': '25', 'neg_90d': '27', 'neg_1yr': '22', 'neg_lifetime': '4', 'ct_30d': '8', 'ct_90d': '22', 'ct_1yr': '41', 'ct_lifetime': '1,309', 'vendor_id': 'ABOS4QKXWWCIJ'}

/gp/aag/main/ref=olp_merch_name_4?ie=UTF8&asin=B01MQZ6EFY&isAmazonFulfilled=0&seller=AAJKNHCJHI47S
{'pos_30d': '86', 'pos_90d': '73', 'pos_1yr': '79', 'pos_lifetime': '78', 'neut_30d': '0', 'neut_90d': '3', 'neut_1yr': '3', 'neut_lifetime': '4', 'neg_30d': '

{'pos_30d': '75', 'pos_90d': '73', 'pos_1yr': '85', 'pos_lifetime': '92', 'neut_30d': '0', 'neut_90d': '0', 'neut_1yr': '0', 'neut_lifetime': '4', 'neg_30d': '25', 'neg_90d': '27', 'neg_1yr': '15', 'neg_lifetime': '5', 'ct_30d': '4', 'ct_90d': '15', 'ct_1yr': '54', 'ct_lifetime': '816', 'vendor_id': 'AR6H8UB58UXM0'}

/gp/aag/main/ref=olp_merch_name_3?ie=UTF8&asin=B01MXS7GVZ&isAmazonFulfilled=0&seller=A1A85P6DGK1T6F
{'pos_30d': '0', 'pos_90d': '75', 'pos_1yr': '85', 'pos_lifetime': '91', 'neut_30d': '100', 'neut_90d': '25', 'neut_1yr': '8', 'neut_lifetime': '2', 'neg_30d': '0', 'neg_90d': '0', 'neg_1yr': '8', 'neg_lifetime': '7', 'ct_30d': '1', 'ct_90d': '4', 'ct_1yr': '13', 'ct_lifetime': '125', 'vendor_id': 'A1A85P6DGK1T6F'}

/gp/aag/main/ref=olp_merch_name_1?ie=UTF8&asin=B00KCXG8R4&isAmazonFulfilled=1&seller=A85DGD18FEXXS
{'pos_30d': '86', 'pos_90d': '81', 'pos_1yr': '85', 'pos_lifetime': '88', 'neut_30d': '3', 'neut_90d': '4', 'neut_1yr': '3', 'neut_lifetime': '2', 'neg_30d': '11', 

{'pos_30d': '86', 'pos_90d': '84', 'pos_1yr': '89', 'pos_lifetime': '94', 'neut_30d': '2', 'neut_90d': '3', 'neut_1yr': '4', 'neut_lifetime': '3', 'neg_30d': '12', 'neg_90d': '12', 'neg_1yr': '8', 'neg_lifetime': '3', 'ct_30d': '58', 'ct_90d': '217', 'ct_1yr': '710', 'ct_lifetime': '6,692', 'vendor_id': 'AS1CTZCHZ2GX6'}

/gp/aag/main/ref=olp_merch_name_5?ie=UTF8&asin=B00UXOLVBA&isAmazonFulfilled=0&seller=A31UAYA3LR7T6J
{'pos_30d': '100', 'pos_90d': '98', 'pos_1yr': '90', 'pos_lifetime': '91', 'neut_30d': '0', 'neut_90d': '0', 'neut_1yr': '1', 'neut_lifetime': '2', 'neg_30d': '0', 'neg_90d': '2', 'neg_1yr': '9', 'neg_lifetime': '7', 'ct_30d': '16', 'ct_90d': '42', 'ct_1yr': '165', 'ct_lifetime': '515', 'vendor_id': 'A31UAYA3LR7T6J'}

/gp/aag/main/ref=olp_merch_name_8?ie=UTF8&asin=B01MQZ6EFY&isAmazonFulfilled=0&seller=A1UYCFJO0ZQRFI
something went wrong when processing feedback_table_data
{'pos_30d': None, 'pos_90d': None, 'pos_1yr': None, 'pos_lifetime': None, 'neut_30d': None, 'neut_90

something went wrong when processing feedback_table_data
{'pos_30d': None, 'pos_90d': None, 'pos_1yr': None, 'pos_lifetime': None, 'neut_30d': None, 'neut_90d': None, 'neut_1yr': None, 'neut_lifetime': None, 'neg_30d': None, 'neg_90d': None, 'neg_1yr': None, 'neg_lifetime': None, 'ct_30d': None, 'ct_90d': None, 'ct_1yr': None, 'ct_lifetime': None, 'vendor_id': 'A1SPB71ACSV813'}

/gp/aag/main/ref=olp_merch_name_2?ie=UTF8&asin=B0762ZDDH5&isAmazonFulfilled=0&seller=A1HGDLL7IC698Q
{'pos_30d': '100', 'pos_90d': '100', 'pos_1yr': '91', 'pos_lifetime': '92', 'neut_30d': '0', 'neut_90d': '0', 'neut_1yr': '1', 'neut_lifetime': '3', 'neg_30d': '0', 'neg_90d': '0', 'neg_1yr': '8', 'neg_lifetime': '5', 'ct_30d': '2', 'ct_90d': '14', 'ct_1yr': '77', 'ct_lifetime': '178', 'vendor_id': 'A1HGDLL7IC698Q'}

/gp/aag/main/ref=olp_merch_name_1?ie=UTF8&asin=B01CGX440U&isAmazonFulfilled=1&seller=A30QOUJJTD820N
{'pos_30d': '83', 'pos_90d': '90', 'pos_1yr': '91', 'pos_lifetime': '91', 'neut_30d': '3', 'neut_90

...

{'pos_30d': '100', 'pos_90d': '100', 'pos_1yr': '99', 'pos_lifetime': '99', 'neut_30d': '0', 'neut_90d': '0', 'neut_1yr': '0', 'neut_lifetime': '0', 'neg_30d': '0', 'neg_90d': '0', 'neg_1yr': '1', 'neg_lifetime': '1', 'ct_30d': '13', 'ct_90d': '20', 'ct_1yr': '91', 'ct_lifetime': '216', 'vendor_id': 'AH3Q1D0WCL82'}

/gp/aag/main/ref=olp_merch_name_1?ie=UTF8&asin=B01LL9TAAO&isAmazonFulfilled=1&seller=A3HOMR1H5AROYT
{'pos_30d': '100', 'pos_90d': '99', 'pos_1yr': '99', 'pos_lifetime': '100', 'neut_30d': '0', 'neut_90d': '0', 'neut_1yr': '0', 'neut_lifetime': '0', 'neg_30d': '0', 'neg_90d': '1', 'neg_1yr': '0', 'neg_lifetime': '0', 'ct_30d': '190', 'ct_90d': '501', 'ct_1yr': '1,418', 'ct_lifetime': '2,137', 'vendor_id': 'A3HOMR1H5AROYT'}

/gp/aag/main/ref=olp_merch_name_3?ie=UTF8&asin=B01I0LDUZM&isAmazonFulfilled=1&seller=A1APD86YI450VM
{'pos_30d': '99', 'pos_90d': '99', 'pos_1yr': '99', 'pos_lifetime': '100', 'neut_30d': '0', 'neut_90d': '0', 'neut_1yr': '0', 'neut_lifetime': '0', 'neg_30

something went wrong when processing feedback_table_data
{'pos_30d': None, 'pos_90d': None, 'pos_1yr': None, 'pos_lifetime': None, 'neut_30d': None, 'neut_90d': None, 'neut_1yr': None, 'neut_lifetime': None, 'neg_30d': None, 'neg_90d': None, 'neg_1yr': None, 'neg_lifetime': None, 'ct_30d': None, 'ct_90d': None, 'ct_1yr': None, 'ct_lifetime': None, 'vendor_id': 'AJA1CWGF2U8OZ'}

/gp/aag/main/ref=olp_merch_name_3?ie=UTF8&asin=B00BWB2ZRW&isAmazonFulfilled=0&seller=A7GPY77XZ4DQV
something went wrong when processing feedback_table_data
{'pos_30d': None, 'pos_90d': None, 'pos_1yr': None, 'pos_lifetime': None, 'neut_30d': None, 'neut_90d': None, 'neut_1yr': None, 'neut_lifetime': None, 'neg_30d': None, 'neg_90d': None, 'neg_1yr': None, 'neg_lifetime': None, 'ct_30d': None, 'ct_90d': None, 'ct_1yr': None, 'ct_lifetime': None, 'vendor_id': 'A7GPY77XZ4DQV'}

/gp/aag/main/ref=olp_merch_name_4?ie=UTF8&asin=B006NOFPGG&isAmazonFulfilled=0&seller=A26TO5OAX3KDFO
something went wrong when processing fe

{'pos_30d': '-', 'pos_90d': '100', 'pos_1yr': '100', 'pos_lifetime': '100', 'neut_30d': '-', 'neut_90d': '0', 'neut_1yr': '0', 'neut_lifetime': '0', 'neg_30d': '-', 'neg_90d': '0', 'neg_1yr': '0', 'neg_lifetime': '0', 'ct_30d': '0', 'ct_90d': '1', 'ct_1yr': '3', 'ct_lifetime': '3', 'vendor_id': 'A256PDPOL7MXSB'}

/gp/aag/main/ref=olp_merch_name_9?ie=UTF8&asin=B0765BB4DW&isAmazonFulfilled=0&seller=AK9CGQP37WB76
{'pos_30d': '-', 'pos_90d': '-', 'pos_1yr': '100', 'pos_lifetime': '100', 'neut_30d': '-', 'neut_90d': '-', 'neut_1yr': '0', 'neut_lifetime': '0', 'neg_30d': '-', 'neg_90d': '-', 'neg_1yr': '0', 'neg_lifetime': '0', 'ct_30d': '0', 'ct_90d': '0', 'ct_1yr': '3', 'ct_lifetime': '3', 'vendor_id': 'AK9CGQP37WB76'}

/gp/aag/main/ref=olp_merch_name_1?ie=UTF8&asin=B01GVDQFQW&isAmazonFulfilled=1&seller=AX7379RXEJ3QE
{'pos_30d': '100', 'pos_90d': '100', 'pos_1yr': '100', 'pos_lifetime': '100', 'neut_30d': '0', 'neut_90d': '0', 'neut_1yr': '0', 'neut_lifetime': '0', 'neg_30d': '0', 'neg_90d

{'pos_30d': '100', 'pos_90d': '100', 'pos_1yr': '100', 'pos_lifetime': '99', 'neut_30d': '0', 'neut_90d': '0', 'neut_1yr': '0', 'neut_lifetime': '1', 'neg_30d': '0', 'neg_90d': '0', 'neg_1yr': '0', 'neg_lifetime': '0', 'ct_30d': '24', 'ct_90d': '75', 'ct_1yr': '321', 'ct_lifetime': '1,128', 'vendor_id': 'AH9ISI08VU3KL'}

/gp/aag/main/ref=olp_merch_name_7?ie=UTF8&asin=B00QOR4WPM&isAmazonFulfilled=0&seller=A33QK8AZPLXN7A
{'pos_30d': '-', 'pos_90d': '100', 'pos_1yr': '100', 'pos_lifetime': '100', 'neut_30d': '-', 'neut_90d': '0', 'neut_1yr': '0', 'neut_lifetime': '0', 'neg_30d': '-', 'neg_90d': '0', 'neg_1yr': '0', 'neg_lifetime': '0', 'ct_30d': '0', 'ct_90d': '1', 'ct_1yr': '12', 'ct_lifetime': '49', 'vendor_id': 'A33QK8AZPLXN7A'}

/gp/aag/main/ref=olp_merch_name_6?ie=UTF8&asin=B00QOR4WPM&isAmazonFulfilled=0&seller=A79Q5A2ZFB97S
{'pos_30d': '100', 'pos_90d': '100', 'pos_1yr': '100', 'pos_lifetime': '98', 'neut_30d': '0', 'neut_90d': '0', 'neut_1yr': '0', 'neut_lifetime': '2', 'neg_30d': 

TypeError: must be str, not float

In [ ]:
# table = soup.find_all('table', {'id': 'feedback-summary-table'})[0]
# rows = table.find_all('tr')
# if len(rows) == 0:
#     error_message('parsing table')
# #     return
# row = rows[0]
# rc = row.findChildren()
# len(rc)

# soup.find_all('span', {'data-action': 'expand-text'})

In [ ]:
# review_rows = soup.find_all('tr', {'class': 'feedback-row'})
# for row in review_rows:
#     try:
#         row_id = row['id']
#         if row_id == 'feedback-row-template':
#             continue
#         else:
#             print(row_id, 'RID')
#     except KeyError:
#         pass
#     rating = row.find_all('span', {'class': 'a-icon-alt'})[0].text
#     print(rating)
#     text = row.find_all('span', {'class': 'a-text-quote'})[0].text
#     print(text)

In [30]:
a_df.head()

,asin,is_amzn_fulf,is_prime_x,name,num_ratings,over_time_span,pct_positive,price,vendor_url,keyword,...,is_prime_y,promo_category,promo_description,is_supersaver_shipping,num_images,manufacturer,parent_asin,upc,currency,vendor_id
0,B000Y7VCZQ,True,True,"6pm, LLC",218941.0,12 months,99.0,24.99,/gp/aag/main/ref=olp_merch_name_1?ie=UTF8&asin...,Tommy Hilfiger Handbags,...,True,NaN,NaN,True,2.0,NaN,B00RZASODS,6.461301e+11,USD,A2BMBHD2OU3XDU
1,B000Y7VCZQ,True,True,Country Farm Resellers,879.0,12 months,100.0,75.99,/gp/aag/main/ref=olp_merch_name_2?ie=UTF8&asin...,Tommy Hilfiger Handbags,...,True,NaN,NaN,True,2.0,NaN,B00RZASODS,6.461301e+11,USD,A3CEO3H9NLIHCU
2,B002CEOJ7W,False,False,JE Accessories,22.0,12 months,86.0,154.99,/gp/aag/main/ref=olp_merch_name_1?ie=UTF8&asin...,Longchamp handbags,...,False,NaN,NaN,False,4.0,NaN,B0043IY2V8,6.711942e+11,USD,A13FESWZEM0A3D
3,B002CEOJ7W,False,False,The Digital Cantina,460.0,12 months,96.0,245.00,/gp/aag/main/ref=olp_merch_name_2?ie=UTF8&asin...,Longchamp handbags,...,False,NaN,NaN,False,4.0,NaN,B0043IY2V8,6.711942e+11,USD,AREG43AW63ZKB
4,B002E4Q7FW,False,False,JE Accessories,22.0,12 months,86.0,114.99,/gp/aag/main/ref=olp_merch_name_1?ie=UTF8&asin...,Longchamp handbags,...,False,NaN,NaN,False,1.0,Longchamp,B016YZLYCC,6.711942e+11,NaN,A13FESWZEM0A3D


In [31]:
a_df.to_csv('data/joined-kw-fmt.csv', index=False)

In [28]:
a_df = pd.read_csv('data/joined-kw-fmt.csv')
a_df['pct_positive'] = a_df['pct_positive'].apply(fix_pct_pos)
a_df['vendor_id'] = a_df['vendor_url'].apply(url_to_vendor_id)
uv_df = a_df[~a_df.vendor_id.duplicated(keep='first')]
uv_df.sort_values(by='pct_positive', inplace=True)
worst_to_best = uv_df['vendor_url'].values

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


c:\users\the-slick\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [ ]:
len(worst_to_best)


In [ ]:
# error asins

# ['62428357',
#  'B00HDQ2566',
#  'B00MFPVSF4',
#  'B00TLX0GRY',
#  'B017WJ64GA',
#  'B01BJ96A4A',
#  'B01GEXBO6K',
#  'B01HLCL7QK',
#  'B01HUOLVB0',
#  'B01J1HXBBM',
#  'B01J2CEZW0',
#  'B01J7GJXUU',
#  'B01LWJ2A8U',
#  'B01LX2AGRM',
#  'B01LY412J5',
#  'B06WVPG71W',
#  'B06XDD6LT5',
#  'B06XQ1WZ8P',
#  'B06XWGMMJW',
#  'B06XWHY2S4',
#  'B06XWZT1LV',
#  'B06Y64R24N',
#  'B0711Y4QG1',
#  'B072QNVQJY',
#  'B0734YS8Q6',
#  'B073VXYN27',
#  'B073VY4P1J',
#  'B073WMJ6XH',
#  'B074GD752Y',
#  'B074NY82W5',
#  'B0752TNCLH',
#  'B0759NP9B5',
#  'B0759SQ4XG',
#  'B0759YT8V7',
#  'B076KK9N4K',
#  'B076RQPW4Q',
#  'B076X43P5Y',
#  'B077P9RXZ6',
#  'B077PFZBB6',
#  'B077S1M36F',
#  'B07848VMC4',
#  'B078VMTFX7',
#  'B079TXGWQR']

In [ ]:
# B014AGD286



# B01J8DF6E4



# B078HW8YTY



# B0787MSV5Z

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B073X5WVSJ



# B0752J9J4X



# B01I3J28MC



# B01M9DUYTX



# B01MRUN093



# B06XWGMMJW



# B01CGX440U



# B0191YBKCC

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B0786M13CN

# No vendor reputation time span found ("97% over 12 months...")


# B01N1P4N1Y

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B01B4Q4QNK



# B01NBTGR4L



# B0779XH742



# B071FG7SKS



# B01NAKGEUY



# B01N5F5RE4

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B06XR81QS9



# B008OT8JWA



# B06XRM3Q1J



# B01MYXNAHU

# No vendor reputation time span found ("97% over 12 months...")


# B01LO9JVNC



# B06XQ4DHL7



# B01GDWWYIE



# B06XX441PJ



# B01KH8NLLY



# B018U2DK2E



# B01N6PWBS1



# B01LSWA8E6



# B01N65DSA8

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B079WG4QV7

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B000Y7VCZQ



# B0781CWG13



# B01NCJAI74



# B076NWNMH1



# B01A5GFGGQ



# B01M8L6Y8U



# B06XWLZM43



# B01KW133P0



# B01DLV87TU



# B00TLX0GRY



# B01H3QNJ60



# B071WTL3SS



# B01NATJG19



# B00NLBGFA4



# B01MS6BE58



# B01DLYCS6A



# B077L2WMPH



# B01MS10QGE

# No vendor reputation time span found ("97% over 12 months...")


# B073X5XX76



# B0771R2593



# B002EVPSX2



# B06XTNCFXC



# B071WRSMZX



# B077V4GG6J

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B0734YS8Q6



# B0728H558R



# B06XNM4LM7



# B06XTVS41D



# B01H0OOAPE



# B01MQRITXM



# B0781CSKRW



# B013ID33VS



# B077WZ42RW



# B01N214IYR



# B077LZTQJR

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B00Q3XM538



# B01GVDQFQW



# B00KLWOPDA



# B076X43P5Y



# B06X9KT43Y



# B01N4V05BQ



# B073X6BN3S



# B01MPX2ZIQ



# B01MU9KSTR



# B078MWHVQZ

# No vendor reputation time span found ("97% over 12 months...")


# B06VW5RMNS

# No vendor reputation time span found ("97% over 12 months...")


# B01MXLE7AB

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B077H4Q8QW

# No vendor reputation time span found ("97% over 12 months...")


# B01HHKA2S0



# B075Y9RR8G



# B0763RWDBM



# B00UXOLVBA



# B01KH8NNDA



# B0725WDRW3



# B078PS6XGQ

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B071J1W8TL



# B0752TNCLH



# B0789RGFWF



# B01HHK9ZVU



# B01KW13MIS



# B01MYXNVWL



# B01M6ZOFOS



# B077H5RVK5

# No vendor reputation time span found ("97% over 12 months...")


# B072NDR754

# No vendor reputation time span found ("97% over 12 months...")


# B01MZ156JF

# No vendor reputation time span found ("97% over 12 months...")


# B077VDCC2G

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B019ICH3PK



# B072FNWF1B



# B074RZHZVM



# B00H4DUK7O



# B06XQL21VJ



# B01N7FF4BS



# B0725XD8T9



# B072J8GRGT



# B074XBQ6V2

# No vendor reputation time span found ("97% over 12 months...")


# B077CXYSP9



# B0759KS4F1



# B01J7GJXUU



# B00LV4W4H6



# B01KW12MK2



# B01GSP120C

# No vendor reputation time span found ("97% over 12 months...")


# B01MFAJNO0



# B077PJ6SXN



# B01MRSWFQO



# B01N0LZUHF



# B01M3RSDIP

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B076VPCS7M



# B01N1XMZGQ

# No vendor reputation time span found ("97% over 12 months...")


# B078GYZ55Y



# B06W2K93RM



# B06XJ2Q4QB



# B01N2U7H5U



# B0773441QG



# B00KCXG8R4



# B00TUV4DPI



# B074HT29NZ

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B01KXUO11K



# B078X4W3H6



# B078PS21TM



# B073VXSQPF



# B01KA1MGN2



# B01MTD0QN8



# B079J626RF

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B01N75EBUR



# B01MSSRIMA



# B06Y572SFR



# B01MS81PSM



# B00BWB2ZRW



# B077VDTBPR



# B01MQZ6EFY

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B074CLTPXL

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B00WDVW8YK



# B076Z1GW69

# No vendor reputation time span found ("97% over 12 months...")


# B01H0SINZ8



# B079KVBDD3

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B0765BB4DW

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")


# B01N4FM6TD



# B01M0E689X



# B077PGW3D6



# B078NKGL3B



# B078WC11YB



# B01G4B59IG



# B06XSY1PST



# B075F8N39Z



# B06WRSNP4M



# B076FDWPRV



# B01M0E8PXA



# B06XTXF9VK



# B019TSR5XI



# B06XW1TQJ2



# B073X5T2LS



# B073X5WVS5



# B06XWHY2S4



# B01NBOTR8V



# B074ZRDBSC



# B079JT2R8T

# No vendor reputation time span found ("97% over 12 months...")


# B004EF5VVA



# B078FGJW68



# B071SDNJPJ



# B079TXH3BZ



# B078VMTFX7



# B01EY70LZ8



# B06XKTK3DH

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B079TXGWQR



# B007206FKA



# B01MRSWGTZ



# B076M9RS85



# B071P77353



# B074XH137F

# No vendor reputation time span found ("97% over 12 months...")


# B06WRPL8TB



# B0728H5P47



# B01MXS7GVZ

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B01N4MYDXP



# B06X6BLJXY



# B00GDCGXS2



# B0787QXD5F



# B06X16QZ1K



# B073WMDRNL



# B01I47LAY0



# B0734XM3BY



# B01KHC6T00



# B01MXLBIEK



# B06XJ7CXGW



# B06XHN4ZW5



# B06XWZT1LV



# B076WYD2Y2



# B06VWMKZ6G



# B0762SSXMY



# B0734YM7GW



# B06XFM2NCG



# B0783NDR8Z



# B06W9J8LHR



# B077PFZBB6



# B01BZ7OXYA



# B00D34ZVF4

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B01N5OASYO



# B01M4M9DEK

# No vendor reputation time span found ("97% over 12 months...")


# B06XQJV1VF



# B01GRPSUQM



# B01N2GYMK5



# B01J7GJQ3E



# B01N49K3ZI

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")


# B01N5PC7OK

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B06XQ4KXXX



# B01N2OQDJ4

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B077PZD8C4



# B00LPHDBEO



# B00QOR4WPM

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B077YYBRMP

# No vendor reputation time span found ("97% over 12 months...")


# B06VWT2D3Q



# B07814CGKG



# B077M8NQNG



# B01N7I6FKF



# B0789SKD5G



# B0752R7MWZ

# No vendor reputation time span found ("97% over 12 months...")


# B071J1XDVF



# B01MAZH0HO



# B06XW194CJ



# B075QWKDRT



# B06ZYX73KR



# B01M01CIIG



# B01M7SINR0

# No vendor reputation time span found ("97% over 12 months...")


# B06W2LHLV2



# B01MTMNSPA



# B079TZHRZS



# B073X5XX6R



# B079KYHC38

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B071SDPTWL



# B016A55T4A



# B077DG3KK1



# B06XZ9PM79



# B073VVHFBF



# B077H3RM44



# B06XTK25ML



# B013G0BYVY



# B01M0PRDEA



# B06XB2J5ZM



# B06XKWZW92



# B06ZZJNDHL



# B01NGZ40MC



# B01GRD6CE6



# B01KHDNLC8



# B073VXDTFH



# B0711Y4QG1



# B0793L8MFH

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B01LWJ2A8U



# B01I1ISON2



# B018TPZV2E



# B06XQK5T7L



# B0752RVZTK



# B01N3B3GN2



# B01N5AUDAH



# B018TPYAR6



# B01J1HXBBM



# B01MY76A2A



# B077PLQ1X3



# B07889MHR7



# B073WMJ6XH



# B014KRW86W



# B0789PL492



# B074KG6C7V



# B072QNTV7C



# B072QNVQJY



# B076KK9N4K



# B0777PLW8L

# No vendor reputation time span found ("97% over 12 months...")


# B077QDLG7C



# B071WTGD1M



# B077QLX219



# B078TFFRGC



# B078YC3WSY



# B0789QCV7S



# B0788QTN59



# B06XB5N6YP



# B06XB4RGR3



# B01KHDNECK



# B074VD3DNM



# B01KHAGIX0



# B07848VMC4



# B077Q4ZSB5



# B077P9RXZ6



# B01KH9Y1DK



# B072M34FQX



# B077P9YK1V



# B072LV64HM

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B071SDPGKK



# B01MF4XMH1



# B076KLLFR5



# B01N22DG49



# B078RCTJ2Q



# B06WGS1BH6



# B06XTWKSQS



# B00Y35ZMSI



# B00HDQ2566



# B073VVHFBP



# B01MRKHYW2



# B0734YNX7C



# B01MS7EIJU



# B06XTVQN94

# No vendor reputation time span found ("97% over 12 months...")


# B06XTWTQG2



# B073XG2DRK



# B01B4Q4UP4



# B071HWT4BC



# B01LY412J5



# B0777GJK53

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B06XDDQ5DY



# B073VY1PYY



# B073WN4LDT



# B077KNFTR1



# B06XD9G8B1



# B073VY4P1J



# B073VXYN27



# B0763SF63H



# B074ZH31L1



# B077J4XRRL



# B073WNH5W6



# B0762ZDDH5



# B07263KZWZ



# B077Z3TWVL

# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B076X1JLGG

l2b ^   ____      non l2b v

In [ ]:
# B01I5R3UBU



# B01D93IANI

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")


# B00MFPVSF4



# B06XBVYBNL



# B075L9S6WT



# B016QUYJEU



# B071G69T8X



# B00UVGTQ1C

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B01KI062GM



# B01KQK2ZLU

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")


# B00NVONA2M

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B075TFVTBX

# No vendor reputation time span found ("97% over 12 months...")


# B06XCCV5FF



# B06VY5NQBL



# B0719GGPKR



# B015NHN574



# B071L48MWJ



# B015CZMIIE



# B0168WJZBI

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B01ENQR806

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")


# B006NOFPGG

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")


# B00O9N8EN4



# B002CEOJ7W



# B00LOUD3I6



# B009AWWI86

# No vendor reputation time span found ("97% over 12 months...")


# B01MRV5U4H



# B00P7PXSA2



# B01HBA2LSK



# B074CH6733



# B01GH9Q042



# B01CYTKXEW



# B01FOC25OW



# B06XSRN6QV



# B009ALJ4NO



# B01H5FMWUI



# B0053T86T0

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B072LF2MPZ



# B016PNJKGA



# B0744G9QBG



# B075DPXC43

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B0759P99WV



# B00HS5OWJA

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B009SOCUQQ



# B01J8UFM8W



# B01JRKTE3M



# B01J4W192W

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")


# B073QH5KCP



# B073QH7N8P



# B01N4KVAAL



# B0759S7V7Z



# B00JLNTOPE

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B011SD7A1O



# B077325Y5Y



# B078B429CJ



# B072169J9N



# B00PWK8YIS

# No vendor reputation time span found ("97% over 12 months...")


# B076RQPW4Q



# B078BBGQ5V



# B01FXXY9U0

# No vendor reputation time span found ("97% over 12 months...")


# B00KLH1MN6

# No vendor reputation time span found ("97% over 12 months...")


# B01LX2AGRM



# B00WXT4IUY

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")


# B01JKZ3390



# B071HXNCJJ



# B012B1J1QE



# B01M24G338

# No vendor reputation time span found ("97% over 12 months...")


# B0757GYZHN



# B018KEA99E



# B00M3OHVGM



# B06XJVFHQF



# B01AUW7EVU



# B01HUOLVB0



# B06XX2P2GH



# B078LMGTT5



# B00WLLOAQ6



# B014F6OE1U



# B01L86PIFG



# B00IYV2JHE



# B00KHGKGZG



# B079DXQJW4



# B0175YO73O



# B072P644XN



# B00O652T2M



# B074GD752Y



# B071VJZBBC



# B01N6S095F



# B079SS4Z8J



# B01N4I6G7H



# B00JIIZCM6



# B06XCD5F34



# B01MSK58AA



# B016QTMBUK



# B01HA9LHOQ



# B01MYDQ7MC



# B01H64JN6O



# B01J2CEZW0



# B0734Q8MJP



# B01N99KJ4B



# B071G4KPN9



# B077S1M36F



# B0759NP9B5



# B0711D6TXD



# B0759SQ4XG



# B06XCC5N19

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B06XDD6LT5



# B01MTPH89M



# B0783NXR75

# No vendor reputation time span found ("97% over 12 months...")


# B01I0LDUZM



# B071YZ9P1K



# B00IK3D04M



# B06VSX5XHX



# B0038MHAN2



# B003IS3HV0



# B075ZYBW77



# B0038MHAP0



# B00SXVPTUO



# B01HLCL7QK



# B01HMT50RO



# B01N52POEE



# B01LL9TAAO



# B076Z89RF3



# B01N64V186



# B074NY82W5



# B06ZYGR2GJ



# B016Z3EFL0



# B06XPTVN6D



# B01KIGXR8M



# B01ACZFSM2



# B06XHTMQVJ



# B002E4Q7FW



# B0788MHVGZ



# B00V9ZY010



# B06Y64R24N



# B075Z6JDSH



# B015U186TK

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")


# B0768N7FDL



# B01FXXYVPI



# B0759Y2G2P



# B0759Y8Z36



# 62428357



# B01GEXBO6K



# B075XRWFL7



# B06WVPG71W



# B0759V381L



# B07198XM1T



# B01FVVOU94



# B06XS9C3DT

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B0759VGHJS



# B01GFVD2OI



# B074NYJNPM

# No vendor reputation time span found ("97% over 12 months...")


# B071NB42VN

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# No vendor reputation time span found ("97% over 12 months...")


# B06Y15H7G4



# B01B85PY9C



# B015QDHL0C

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B0759YT8V7



# B01MPYIWTE



# B06XQ1WZ8P



# B01ITHNX0E



# B079JHVXBQ



# B076LNPNQ7



# B01AUW7FQO



# B00UVH247O

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B00EJWQ46O

# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings
# No vendor reputation time span found ("97% over 12 months...")
# something went wrong when processing num_ratings


# B076FTWGZR



# B01MY1FKZQ



# B076117PRT



# B0764HCNWB



# B01MY1EU94

# No vendor reputation time span found ("97% over 12 months...")


# B06Y6JHBM5



# B01NAJ1F76



# B01BJ96A4A



# B072QZ5RYP



# B073MJ63FJ



# B076579S9G